# ADES Calling Workspace API Flow

In [ ]:
import requests
import urllib3

urllib3.disable_warnings()

domain = "demo.eoepca.org"
auth_server = f"auth.{domain}"
ades_server = f"ades.{domain}"

## Client (e.g. Portal) authenticates user and calls the ADES

In [ ]:
client_id = "e2ca0359-c961-496d-8365-bdaa409b16f3"
client_secret = "9c865400-d976-45db-9929-bba26ba64af1"

### Get Token Endpoint from OIDC Configuration

In [ ]:
# Get OIDC Configuration
oidc_config_endpoint = f"https://{auth_server}/.well-known/openid-configuration"
headers = {"accept": "application/json" }
oidc_config = requests.get(oidc_config_endpoint, headers=headers).json()

# Extract the token endpoint
token_endpoint = oidc_config["token_endpoint"]

### Get Tokens for User `eric`

In [ ]:
eric_name = "eric"
eric_password = "defaultPWD"
headers = { "cache-control": "no-cache" }
data = {
    "scope": "openid user_name is_operator",
    "grant_type": "password",
    "username": eric_name,
    "password": eric_password,
    "client_id": client_id,
    "client_secret": client_secret
}
token_response = requests.post(token_endpoint, headers=headers, data=data).json()
eric_id_token = token_response["id_token"]
eric_access_token = token_response["access_token"]
eric_refresh_token = token_response["refresh_token"]

### Call ADES as user `eric` using the ID token

In [ ]:
# ADES endpoint
ades_eric_endpoint = f"https://{ades_server}/eric/wps3/processes"

In [ ]:
# Request: List Processes
headers = { "accept": "application/json", "Authorization": f"Bearer {eric_id_token}" }
process_list_response = requests.get(ades_eric_endpoint, headers=headers).json()
for process in process_list_response["processes"]:
    print(process["id"])

## ADES as Client calling the Workspace API

The ADES will be called by the above client, that supplies the token to the ADES in `Authorization: Bearer` header - e.g. in execute request.

We consider two cases...
1. ADES simply reuses the token received by including it in `Authorization: Bearer` header in calls it makes to the Workspace API.<br>
  _In this case the ADES is passive and does not act as a client of the authorization server._
2. ADES uses the received ID Token to follow a UMA flow between itself, the resource server (workspace API in this case) and the authorization server.<br>
  The outcome of this is that the ADES obtains an access token (using the ID Token), at the time it is needed.<br>
  _In this case the ADES is a client of the authorization server, and has associated client credentials.<br>
  This is effectively what is currently performed by the resource-guard (uma-user-agent+PEP) in the develop/demo EOEPCA deployments using Gluu._

### Receive token from incoming request
Token was createdby/for a different client

In [ ]:
ades_rx_token = eric_id_token

### CASE 1: Use RX token to call the Workspace API

In [ ]:
# Workspace API endpoint
uri_for_request = f"/workspaces/demo-user-{eric_name}"
workspace_api_eric_endpoint = f"https://workspace-api.{domain}{uri_for_request}"

In [ ]:
# Request: Get Workspace Details
headers = { "accept": "application/json", "Authorization": f"Bearer {ades_rx_token}" }
get_eric_workspace_response = requests.get( workspace_api_eric_endpoint, headers=headers ).json()
get_eric_workspace_response

### CASE 2: Create own token via UMA Flow

The current demo cluster uses the resource-guard to protect resource servers, such as the workspace-api<br>
This would interfere with our ability to follow the UMA flow, since the resource-guard does this for us.

Therefore, we have to bypass the UMA client of the resource-guard, by going directly to the PEP API that authorizes access to the workspace API.<br>
In this context the PEP API of the workspace-api acts as the resource server endpoint for the flow.

#### We need to use the pep API endpoint which acts on behalf of the workspace service

In [ ]:
workspace_pep_api = f"https://workspace-api-pep.{domain}/authorize"

#### ADES Client Credentials (different from other client)

In [ ]:
ades_client_id = "cf394398-5c2e-4580-a67a-6bcc2862f2d6"
ades_client_secret = "70d42655-b1f1-4093-98c1-5d0fd3a23b0a"

#### First we make a 'naive' request to resource with no token, ticket returned

The naive attempt is made to the resource server (workspace API) which returns a ticket in consultation with the authorization server.<br>
Remember that the PEP API is acting as the resource server endpoint - as our workaround the resource-guard.

In [ ]:
headers = {
    "content-type": "application/json", "cache-control": "no-cache",
    "X-Original-Uri": uri_for_request, "X-Original-Method": "GET"
}
naive_attempt_response = requests.get(workspace_pep_api, headers=headers, verify=False)

The ticket is returned in the `WWW-Authenticate` header of the `401 Unauthorized` response...

In [ ]:
if naive_attempt_response.status_code == 401:
    ticket = naive_attempt_response.headers["WWW-Authenticate"].split("ticket=")[1]
    print(f"ticket: {ticket}")
else:
    print("UNEXPECTED: was expecting a 401 response with a ticket")

#### Then Exchange ticket for access token using the original ID Token

In [ ]:
headers = { "cache-control": "no-cache" }
data = {
    "claim_token_format": "http://openid.net/specs/openid-connect-core-1_0.html#IDToken",
    "claim_token": ades_rx_token,
    "ticket": ticket,
    "grant_type": "urn:ietf:params:oauth:grant-type:uma-ticket",
    "client_id": ades_client_id,
    "client_secret": ades_client_secret,
    "scope": "openid"
}
exchange_ticket_response = requests.post(token_endpoint, headers=headers, data=data).json()
eric_access_token_from_ticket = exchange_ticket_response["access_token"]
eric_access_token_from_ticket

#### Check that the new access token works

As a quick check, we can repeat the call we made to the resource server (PEP API) - this time using the token we obtained...

In [ ]:
headers = {
    "content-type": "application/json", "cache-control": "no-cache",
    "X-Original-Uri": uri_for_request, "X-Original-Method": "GET",
    "Authorization": f"Bearer {eric_access_token_from_ticket}"
}
check_token_response = requests.get(workspace_pep_api, headers=headers, verify=False)

We expect a `200 OK` response this time - rather than the `401` we got with the naive attempt...

In [ ]:
check_token_response

#### Make request to Workspace API using new token

As an additional check, we can also use this token with the workspace-api, accessed via the resource-guard.<br>
In this case, by presenting the newly generated access token in the request, the resource-guard does not need to follow the UMA flow - the access token is enough, i.e. we already did the hard work by following the UMA flow ourselves.

In [ ]:
headers = {
    "accept": "application/json",
    "Authorization": f"Bearer {eric_access_token_from_ticket}"
}
get_eric_workspace_response = requests.get( workspace_api_eric_endpoint, headers=headers )
print(get_eric_workspace_response)
get_eric_workspace_response.json()

# Keycloak

In [ ]:
import requests
import urllib3

urllib3.disable_warnings()

realm = "master"
domain = "develop.eoepca.org"
auth_server = f"https://identity.keycloak.{domain}"
dummy_service = f"https://identity.dummy-service.{domain}"
identity_api = f"https://identity.api.{domain}"

token_endpoint = requests.get(f"{auth_server}/realms/{realm}/.well-known/openid-configuration").json()['token_endpoint']

#### Get Eric tokens
Get tokens using eoepca-portal client since that is a frontend client where users can sign in

In [ ]:
eric_tokens = requests.post(token_endpoint, data={
    "grant_type": "password",
    "client_id": "eoepca-portal",
    "username": "eric",
    "password": "eric",
    "scope": "openid"
}).json()
eric_access_token = eric_tokens['access_token']
eric_id_token = eric_tokens['id_token']

#### Get RPT using Eric's access token with a permission to 'view' /ericspace/* resource

In [ ]:
headers = {
    "Authorization": "Bearer " + eric_access_token
}
data = {
    "grant_type": "urn:ietf:params:oauth:grant-type:uma-ticket",
    "audience": "dummy-service",
    "permission": "/ericspace/*#view",
    "permission_resource_format": "uri"
}
reply = requests.post(token_endpoint, headers=headers, data=data).json()
rpt = None
if not 'access_token' in reply:
    print(str(reply))
else:
    rpt = reply['access_token']
    print("Eric's RPT to access /ericspace/* resource on dummy-service:")
    print(rpt)

#### Attempt to get RPT using Eric's id token
Id token is rejected by Keycloak.

In [ ]:
headers = {
    "Authorization": "Bearer " + eric_id_token
}
data = {
    "grant_type": "urn:ietf:params:oauth:grant-type:uma-ticket",
    "audience": "dummy-service",
    "permission": "/ericspace/*#view",
    "permission_resource_format": "uri"
}
reply = requests.post(token_endpoint, headers=headers, data=data).json()
print(reply)

#### Access /ericspace using access token
Request is successful since Gatekeeper performs the UMA flow itself.

In [ ]:
headers = {
    "Authorization": "Bearer " + eric_access_token
}
print("Request to " + dummy_service + "/ericspace using access token:")
response = requests.get(dummy_service + "/ericspace", headers=headers)
print(response.status_code)

#### Access /ericspace using RPT
Request should be successful with 200. Not sure if Gatekeeper still performs the UMA flow or is able to pick the permissions section already included on RPT.

In [ ]:
headers = {
    "Authorization": "Bearer " + rpt
}
print("Request to " + dummy_service + "/ericspace using RPT:")
response = requests.get(dummy_service + "/ericspace", headers=headers)
print(response.status_code)

#### Access /alicespace using the same RPT
Reply should be 401.

In [ ]:
headers = {
    "Authorization": "Bearer " + rpt
}
print("Request to " + dummy_service + "/alicespace using RPT:")
response = requests.get(dummy_service + "/alicespace", headers=headers)
print(response.status_code)

## Use offline access to get a new access token 
This showcase demonstrates how offline_access scope can generate a long-lived refresh token which can be used for offline access. Offline tokens live forever if there is at least one use every 30 days, but can also be destroyed after one use depending on configuration. Offline tokens should be used with caution for security reasons. A user can destroy their offline tokens using Keycloak's UI console. 

#### Get Eric's refresh token

In [ ]:
eric_tokens = requests.post(token_endpoint, data={
    "grant_type": "password",
    "client_id": "eoepca-portal",
    "username": "eric",
    "password": "eric",
    "scope": "openid"
}).json()
refresh_token = eric_tokens['refresh_token']

#### Get Eric's offline refresh token
Difference from the previous call is the 'offline_access' scope.

In [ ]:
eric_tokens = requests.post(token_endpoint, data={
    "grant_type": "password",
    "client_id": "eoepca-portal",
    "username": "eric",
    "password": "eric",
    "scope": "openid offline_access"
}).json()
offline_refresh_token = eric_tokens['refresh_token']

#### Get new a new access token right away to confirm it works
Notice the client_id which must be the original client_id who the refresh token was issued to.

In [ ]:
eric_tokens =  requests.post(token_endpoint, data={
    "grant_type": "refresh_token",
    "client_id": "eoepca-portal",
    "refresh_token": refresh_token
}).json()
print("Eric's access token using refresh token:")
print(eric_tokens['access_token'])

eric_tokens = requests.post(token_endpoint, data={
    "grant_type": "refresh_token",
    "client_id": "eoepca-portal",
    "refresh_token": offline_refresh_token
}).json()
print()
print("Eric's access token using offline refresh token:")
print(eric_tokens['access_token'])

#### Try to get a new access token using refresh token with a different client id
A different client_id will be rejected by Keycloak.

In [ ]:
reply = requests.post(token_endpoint, data={
    "grant_type": "refresh_token",
    "client_id": "ades",
    "refresh_token": refresh_token
}).json()
print("Error message:")
print(reply)

#### Expire the regular refresh token
Sleep for 31 minutes to expire the regular refresh token.

In [ ]:
import time

time.sleep(31 * 60)

#### Get new access tokens using both refresh tokens

In [ ]:
print("Getting new access token with regular refresh token")
reply = requests.post(token_endpoint, data={
    "grant_type": "refresh_token",
    "client_id": "eoepca-portal",
    "refresh_token": refresh_token
}).json()
print("Error message:")
print(reply)

print("Getting new access token with offline refresh token")
eric_tokens = requests.post(token_endpoint, data={
    "grant_type": "refresh_token",
    "client_id": "eoepca-portal",
    "refresh_token": offline_refresh_token
}).json()
print("Eric's access token:")
eric_access_token = eric_tokens['access_token']
print(eric_access_token)

#### Access dummy service using the newly generated access token
Result should be 200

In [ ]:
headers = {
    "Authorization": "Bearer " + eric_access_token
}
print("Request to " + dummy_service + "/ericspace using the acces token generated from the offline token:")
response = requests.get(dummy_service + "/ericspace", headers=headers)
print(response.status_code)